<a href="https://colab.research.google.com/github/edgasanc85/AzureOpenAI/blob/main/ChatGPT_El_Camino_a_Cristo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



```
# Tiene formato de código
```

# ChatBot basado en el libro EL CAMINO A CRISTO


## Instalar librerías principales y configuración de API Key de OpenAI

In [1]:
%%capture
!pip install langchain pypdf openai chromadb tiktoken

In [16]:
%%capture
!pip install langchain-openai

In [18]:
from getpass import getpass
import os
import openai

openai.api_type = "azure"
openai.api_base = "https://edgasancdemo.openai.azure.com/"
openai.api_version = "2023-07-01-preview"

openai.api_key = getpass('Enter the secret value: ')
os.environ['OPENAI_API_KEY'] = openai.api_key

os.environ["AZURE_OPENAI_ENDPOINT"] = openai.api_base
os.environ["AZURE_OPENAI_API_KEY"] = openai.api_key



Enter the secret value: ··········


## Carga de documents

In [19]:
import requests
from langchain.document_loaders import PyPDFLoader

urls = [
    'https://www.adven7.org/books/pdf/el_camino_a_cristo.pdf'
]

ml_papers = []

for i, url in enumerate(urls):
    response = requests.get(url)
    filename = f'paper{i+1}.pdf'
    with open(filename, 'wb') as f:
        f.write(response.content)
        print(f'Descargado {filename}')

        loader = PyPDFLoader(filename)
        data = loader.load()
        ml_papers.extend(data)

# Utiliza la lista ml_papers para acceder a los elementos de todos los documentos descargados
print('Contenido de ml_papers:')
print()

Descargado paper1.pdf
Contenido de ml_papers:



In [20]:
type(ml_papers), len(ml_papers), ml_papers[6]

(list,
 144,
 Document(page_content='7\nque está en el seno del Padre, él le ha dado a conocer.”  (San\nJuan 1: 18). “Ni al Padre conoce nadie, sino el Hijo, y\naquel a quien el Hijo lo quisiere revelar.” (San Mateo 11:\n27). Cuando uno de sus discípulos le dijo: “Muéstranos al\nPadre”, Jesús respondió: “Tanto tiempo hace que estoy con\nvosotros, ¿y todavía no me conoces, Felipe? El que me ha\nvisto a mí, ha visto al Padre: ¿Cómo pues dices tú:\nMuéstranos al Padre?” (San Juan 14: 8, 9).\nJesús dijo, describiendo su misión terrenal: “Jehová me\nha ungido para anunciar buenas nuevas a los pobres; me a\nenviado para proclamar a los cautivos, y a los ciegos recobro\nde la vista, para poner en libertad a los oprimidos...” (San\nLucas 4: 18), esta era su obra. Pasó su vida haciendo bien y\nsanando a todos los oprimidos de Satanás.\nHabía aldeas enteras donde no se oía un gemido de\ndolor en casa alguna, porque él había pasado por ellas y\nsanado a todos sus enfermos. Su obra demostraba su d

## Split de documents

In [21]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size= 1500,
    chunk_overlap= 200, #caracteres que se repiten en el fragmento pasado y siguiente para que tenga continuidad
    length_function= len
    )

documents = text_splitter.split_documents(ml_papers)

In [22]:
len(documents), documents[10]

(278,
 Document(page_content='7\nque está en el seno del Padre, él le ha dado a conocer.”  (San\nJuan 1: 18). “Ni al Padre conoce nadie, sino el Hijo, y\naquel a quien el Hijo lo quisiere revelar.” (San Mateo 11:\n27). Cuando uno de sus discípulos le dijo: “Muéstranos al\nPadre”, Jesús respondió: “Tanto tiempo hace que estoy con\nvosotros, ¿y todavía no me conoces, Felipe? El que me ha\nvisto a mí, ha visto al Padre: ¿Cómo pues dices tú:\nMuéstranos al Padre?” (San Juan 14: 8, 9).\nJesús dijo, describiendo su misión terrenal: “Jehová me\nha ungido para anunciar buenas nuevas a los pobres; me a\nenviado para proclamar a los cautivos, y a los ciegos recobro\nde la vista, para poner en libertad a los oprimidos...” (San\nLucas 4: 18), esta era su obra. Pasó su vida haciendo bien y\nsanando a todos los oprimidos de Satanás.\nHabía aldeas enteras donde no se oía un gemido de\ndolor en casa alguna, porque él había pasado por ellas y\nsanado a todos sus enfermos. Su obra demostraba su divina\n

## Embeddings e ingesta a base de datos vectorial

In [24]:
from langchain_openai import AzureOpenAIEmbeddings
from langchain.vectorstores import Chroma

embeddings = AzureOpenAIEmbeddings(
    azure_deployment="demo_embedding-ada",
    openai_api_version="2023-07-01-preview",
)

vectorstore = Chroma.from_documents(
    documents=documents,
    embedding=embeddings
)

retriever = vectorstore.as_retriever(
    search_kwargs = {"k": 3}
    )

## Modelos de chat y cadenas para consulta de información

In [28]:
from langchain.chains import RetrievalQA
from langchain_openai import AzureChatOpenAI

chat = AzureChatOpenAI(
    openai_api_version="2023-07-01-preview",
    azure_deployment="demo_gpt35",
    temperature=0.0
)

qa_chain = RetrievalQA.from_chain_type(
    llm=chat,
    chain_type="stuff",
    retriever=retriever
)

In [29]:
query = "cual es la mayor necesidad del hombre?"
qa_chain.run(query)

'Según el texto proporcionado, la mayor necesidad del hombre es la comunión con Dios a través de la oración y la obediencia a sus requerimientos divinos. El hombre fue creado en armonía con Dios, pero su desobediencia lo alejó de Él y lo hizo cautivo de Satanás. Por lo tanto, la necesidad urgente del hombre es volver a estar en comunión con Dios y encontrar gozo en su presencia.'

In [30]:
query = "por qué debo nacer de nuevo?"
qa_chain.run(query)

'De acuerdo con la Biblia, es necesario nacer de nuevo para ser hecho participante de la vida que Cristo vino a dar y para ver el reino de Dios. El cambio de corazón por el cual somos hechos hijos de Dios se llama nacimiento, y se compara con la germinación de la buena semilla sembrada por el labrador. Los que están recién convertidos a Cristo son como "niños recién nacidos", "creciendo" a la estatura de hombres en Cristo Jesús. Es solamente mediante la vida de Dios como se engendra la vida espiritual en el corazón de los hombres.'

In [31]:
query = "cómo profundizar nuestro caminar con Dios?"
qa_chain.run(query)

'El texto proporciona algunas sugerencias para profundizar nuestro caminar con Dios. Una de ellas es asociarnos con otros cristianos y hablar del amor de Dios y las verdades de la redención. También se menciona la importancia de aprender diariamente más de nuestro Padre celestial y obtener una nueva experiencia de su gracia. Además, se sugiere que debemos pensar y hablar más de Jesús y menos de nosotros mismos para tener más de su presencia. La meditación en la perfección del Salvador también es mencionada como una forma de desear ser transformados y renovados conforme a su imagen. Por último, se destaca la importancia de la oración sincera y tener un espíritu de amor y perdón en nuestro propio corazón al pedir misericordia y bendición de Dios.'

In [32]:
query = "como orar para que las oraciones sean contestadas?"
qa_chain.run(query)

'Según el texto, hay ciertas condiciones que debemos cumplir para que nuestras oraciones sean contestadas. En primer lugar, debemos sentir la necesidad de la ayuda de Dios y buscarlo con humildad y fe. También es importante confesar nuestros pecados y arrepentirnos de ellos para que nuestras oraciones sean aceptadas. Debemos perdonar a los demás y ser perseverantes en la oración, manteniendo una unión continua del alma con Dios. Además, debemos estar abiertos a la influencia del Espíritu y tener un corazón dispuesto a recibir las bendiciones de Dios. En resumen, debemos buscar a Dios con sinceridad, humildad y fe, y cumplir las condiciones de aceptación para que nuestras oraciones sean contestadas.'

In [33]:
query = "Como se hara justo un pecador?"
qa_chain.run(query)

'Según el contexto proporcionado, un pecador puede ser hecho justo a través de la gracia salvadora de Cristo. Cristo vivió una vida impecable, murió por nosotros y ofrece quitarnos nuestros pecados y vestirnos de su justicia. Si nos entregamos a él y lo aceptamos como nuestro Salvador, por pecaminosa que haya sido nuestra vida, seremos contados entre los justos por consideración a él. El carácter de Cristo toma el lugar del nuestro, y somos aceptados por Dios como si no hubiéramos pecado. Además, Cristo cambia el corazón y habita en nuestro corazón por la fe. Debemos mantener esta comunión con Cristo por la fe y la sumisión continua de nuestra voluntad a él.'